In [ ]:
import os
import json
import numpy as np

from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.data.rmg import RMGDatabase
from rmgpy.species import Species
from rmgpy import settings

# Load the Mechanism Uncertainty Information

In [1]:
# Make sure you've loaded:
# RMG-Py/save_sensitivity (now you have to rebuild because you turned word-wrap off in chemkin)
# RMG-database/bm_fit_sensitivity

# Load an example ethane model generated using the trees
from rmgpy.tools.uncertainty import Uncertainty, get_rmg_rxn_from_yaml_rxn

chem = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/chem_annotated.inp'
spec = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/species_dictionary.txt'
uncertainty = Uncertainty(output_directory='./temp/uncertainty')
uncertainty.load_model(chem, spec)

In [2]:
# Load the database used to generate the model
uncertainty.load_database(
    thermo_libraries=['BurkeH2O2', 'CurranPentane','FFCM1(-)','primaryThermoLibrary', 'thermo_DFT_CCSDTF12_BAC', 'DFT_QCI_thermo', 'CBS_QB3_1dHR'],
    kinetics_families='default',
    reaction_libraries=['CurranPentane','FFCM1(-)','combustion_core/version5'],
    kinetics_depositories=['training'],
)


In [3]:
# Get the different reaction (and thermo- don't forget thermo) sources
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties()

In [4]:
import json
def unpack_sensitivity(long_desc):
    start_str = 'sensitivities = '
    if start_str not in long_desc:
        return []
    start_index = long_desc.find(start_str) + len(start_str)
    sensitivities_str = long_desc[start_index:].replace("'", '"')
    sensitivities_str = sensitivities_str.replace("nan", '"-9999999"')
    return json.loads(sensitivities_str)

In [5]:
# List the reactions that came from nodes on the new trees and collect the families
print('These reactions were estimated using the new trees:')

families_to_lookup = set()

empty_derivatives = 0
for reaction_key in uncertainty.reaction_sources_dict.keys():
    if 'Rate Rules' in uncertainty.reaction_sources_dict[reaction_key].keys():
        family = reaction_key.family
        src = uncertainty.reaction_sources_dict[reaction_key]['Rate Rules'][1]
        if src['node'] != '':
            # look up the node name in the database
            if src['node'] not in uncertainty.database.kinetics.families[family].rules.entries.keys():
                print('NODE NOT FOUND\t', src['node'])
                continue
            
            entry = uncertainty.database.kinetics.families[family].rules.entries[src['node']][0]
            derivatives = unpack_sensitivity(entry.long_desc)
            if derivatives == []:
                # It's okay- I checked and I just haven't generated the derivatives for those families yet
                continue
            
            print(f'{reaction_key}\t({family}) \n\t{src["node"]}')
            print()
            
            # collect the set of families to lookup
            families_to_lookup.add(str(family))


print(families_to_lookup)


These reactions were estimated using the new trees:
C2H5(44) + C5H11(421) <=> C7H16(1)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R

NC3H7(93) + PC4H9(183) <=> C7H16(1)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R

H(14) + C7H15(682) <=> C7H16(1)	(R_Recombination) 
	Root_1R->H_N-2R->S_N-2CHNO->H_N-2CNO-inRing_Ext-2CNO-R_N-Sp-3R!H=2CCNNOO_N-2CNO->O_Ext-2CN-R

C2H5(44) + C7H15(682) <=> C2H4(11) + C7H16(1)	(Disproportionation) 
	Root_N-4R->H_4CNOS-u1_N-1R!H->O_N-4CNOS->O_Ext-4CNS-R_N-Sp-5R!H#4CCCNNNSSS_N-2R!H->S_N-5R!H->O_Sp-5CS-4CCNSS_Ext-4CNS-R

NC3H7(93) + C7H15(682) <=> C3H6(12) + C7H16(1)	(Disproportionation) 
	Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C

O2(2) + C7H15(682) <=> S(763)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_1COS->O_Ext-1O-R_E

	Root_Ext-1R!H-R_N-4R->O

OCHO(29) + S(1342) <=> CO2(7) + C7H14(754)	(Disproportionation) 
	Root_Ext-1R!H-R_N-4R->O

C5H11(421) + S(1342) <=> C5H10(424) + C7H14(808)	(Disproportionation) 
	Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C

C5H11(422) + S(1342) <=> C5H10(424) + C7H14(808)	(Disproportionation) 
	Root_Ext-2R!H-R_2R!H->C_4R->C

C5H11(422) + S(1342) <=> C5H10(425) + C7H14(808)	(Disproportionation) 
	Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C

C5H11(421) + S(1342) <=> C5H10(424) + C7H14(754)	(Disproportionation) 
	Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C

C5H11(422) + S(1342) <=> C5H10(424) + C7H14(754)	(Disproportionation)

In [6]:
print('# Node\tTraining Reaction\t\tdA_node/dA_train\tdE0_node/dA_train\tdn_node/dA_train')
for family in families_to_lookup:
    templateRxnMap = uncertainty.database.kinetics.families[family].get_reaction_matches(thermo_database=uncertainty.database.thermo, remove_degeneracy=True,
                                                                             get_reverse=True, exact_matches_only=False, fix_labels=True)
    
    for r, mechanism_reaction in enumerate(uncertainty.reaction_sources_dict.keys()):
        if mechanism_reaction.__class__.__name__ == 'PDepReaction':
            continue
        if mechanism_reaction.family != family:
            continue
            
        if 'Rate Rules' in uncertainty.reaction_sources_dict[mechanism_reaction].keys():
            # family = reaction_key.family
            src = uncertainty.reaction_sources_dict[mechanism_reaction]['Rate Rules'][1]
            node_name = src['node']
            if node_name != '':
                print(f'{mechanism_reaction.index}\t {mechanism_reaction} \n{node_name}')
                # look up the node name in the database
                if node_name not in uncertainty.database.kinetics.families[family].rules.entries.keys():
                    print('DERIVATIVE NOT YET COMPUTED\t',src['node'])
                    continue
            
                entry = uncertainty.database.kinetics.families[family].rules.entries[node_name][0]
                derivatives = unpack_sensitivity(entry.long_desc)
                
                for i, rxn in enumerate(templateRxnMap[node_name]):
                    N = 35
                    pad_length = N - len(str(rxn))
                    rxn_str = str(rxn) + " " * pad_length
                    #compare names from both sources to check order
                    #print("\n\t", derivatives[i]['name'],"\n\t", rxn_str)
                    dA = derivatives[i]['dA']
                    dE0 = derivatives[i]['dE0']
                    dn = derivatives[i]['dn']
                    if (dn == "-9999999"):
                        dn = np.nan
                    print('\t', rxn_str, "\t\t{0:.6f}".format(dA),
                          "\t\t{0:.6f}".format(dE0),
                          "\t\t{0:.6f}".format(dn))
                    print()
            else:
                print('No node for reaction ', mechanism_reaction) # this needs to complain more loudly
            
    
    
    

# Node	Training Reaction		dA_node/dA_train	dE0_node/dA_train	dn_node/dA_train
200	 C2H5(44) + C5H11(421) <=> C7H16(1) 
Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R
	 C3H3 + C7H7 <=> C10H10              		0.089797 		0.000000 		-0.000040

	 C3H3 + C3H3 <=> C6H6-3              		0.089975 		0.000000 		0.000082

	 C3H5 + C3H5 <=> C6H10-2             		0.003108 		0.000000 		0.000167

	 C3H5 + C2H5 <=> C5H10-2             		0.003108 		0.000000 		0.000167

	 C3H5 + CH3 <=> C4H8-2               		0.003107 		0.000000 		0.000166

	 C11H23 + CH3 <=> C12H26             		0.249477 		0.000000 		-0.000083

	 C10H21 + C2H5 <=> C12H26-2          		0.003108 		0.000000 		0.000167

	 C9H19 + C3H7 <=> C12H26-3           		0.003105 		0.000000 		0.000165

	 C8H17 + C4H9-2 <=> C12H26-4         		0.003108 		0.000000 		0.000167

	 C7H15 + C5H11 <=> C12H26-5          		0.046068 		0.000000 		0.000154

	 C6H13 + C6H13 <=> C12H26-6          		0.003107 		0.000000 		0.000166

	

NameError: name 'np' is not defined

# Save the Mechanism Uncertainty Information

In [7]:
import yaml
import cantera as ct
import pickle

In [8]:
# see if we can use the reaction as the dictionary key
mechanism_yaml = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/nheptane.yaml'
gas1 = ct.Solution(mechanism_yaml)
len(gas1.reactions())

2266

In [ ]:
# Read in the yaml mechanism file
with open(mechanism_yaml, 'r') as file:
    mech = yaml.load(file, Loader=yaml.FullLoader)
    # print(mech)

In [16]:
for i, reaction in enumerate(mech['reactions']):
    rmg_rxn = get_rmg_rxn_from_yaml_rxn(uncertainty.reaction_list, reaction)
    # print(uncertainty.reaction_sources_dict[rmg_rxn])
    
    rxn_uncertainty = {}
    # Note the source- assume only one source per reaction, or else this might overwrite existing reaction
    for src_key in uncertainty.reaction_sources_dict[rmg_rxn].keys():
        rxn_uncertainty['source'] = src_key
        
    
    mech['reactions'][i]['uncertainty'] = rxn_uncertainty
    if i > 3000:
        break

In [17]:
output_path = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_uncertainty.yaml'
with open(output_path, 'w') as file:
    output = yaml.dump(mech, file)
    print(output)

None


In [ ]:
# check for duplicate names - yes there are duplicate equations
reaction_names = []
reaction_IDs = []
for reaction_key in uncertainty.reaction_sources_dict.keys():
    try:
        out_reaction = reaction_key.to_cantera()
        if type(out_reaction) is list:
            out_reaction = out_reaction[0]
        if out_reaction.equation in reaction_names:
            print(f'Duplicate: {out_reaction.equation}')
        reaction_names.append(out_reaction.equation)
        reaction_IDs.append(out_reaction.ID)
    except AttributeError:
        print(f'Failed to convert {reaction_key}')
        print(reaction_key.kinetics)
        # raise
print('\nReaction equations')
print(len(reaction_names))
print(len(set(reaction_names)))
print('\nReaction IDs')
print(len(reaction_IDs))
print(len(set(reaction_IDs)))

In [ ]:
print(len(uncertainty.kinetic_input_uncertainties))
print(len(uncertainty.reaction_sources_dict.keys()))

In [ ]:
#print(mech['reactions'][0])
for i, reaction_key in enumerate(uncertainty.reaction_sources_dict.keys()):
    j = reaction_key.index
    try:
        rxn = reaction_key.to_cantera()
        if type(rxn) is not list:
            rxn = [rxn]
        for r in rxn:
            print(j, r.ID)
            print(reaction_key)
            print()
    except AttributeError:
        print(f'failed to convert {reactions_key}')
    if i>10:
        break
#for rxn in mech['reactions']:
#    print(rxn['equation'])
#    print(uncertainty.reactions)